In [42]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

# Check if using GPU

In [43]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


# Load the data

In [44]:
train_data = pd.read_csv('./train.csv')
test_data = pd.read_csv('./test.csv')

# Handle missing values

In [45]:
# Iterating over train_data and test_data
for dataset in [train_data, test_data]:
    # Replacing missing 'Age' values with mean age
    dataset['Age'] = dataset['Age'].fillna(dataset['Age'].mean())
    # Replacing missing 'Embarked' values with 'S'
    dataset['Embarked'] = dataset['Embarked'].fillna('S')
    # Replacing missing 'Fare' values with mean fare
    dataset['Fare'] = dataset['Fare'].fillna(dataset['Fare'].mean())

In [46]:
for column in ['Sex', 'Embarked']:
    le = LabelEncoder()
    # Encoding 'Sex' and 'Embarked' columns in train_data
    train_data[column] = le.fit_transform(train_data[column])
    # Encoding 'Sex' and 'Embarked' columns in test_data
    test_data[column] = le.transform(test_data[column])

In [47]:
scaler = StandardScaler()

# Features to be scaled
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']

# Scaling the features in the train_data
train_data[features] = scaler.fit_transform(train_data[features])
test_data[features] = scaler.transform(test_data[features])

In [48]:
# Creating the test and train features and labels
train_features, val_features, train_labels, val_labels = train_test_split(
    train_data[features], train_data['Survived'], test_size=0.2, random_state=42)

In [49]:
# Converting the test and train features and labels to tensors
X_train = torch.tensor(train_features.values, dtype=torch.float32).to(device)
y_train = torch.tensor(train_labels.values, dtype=torch.long).to(device)
X_val = torch.tensor(val_features.values, dtype=torch.float32).to(device)
y_val = torch.tensor(val_labels.values, dtype=torch.long).to(device)
X_test = torch.tensor(test_data[features].values, dtype=torch.float32).to(device)

In [50]:
# class defining a neural network architecture for the Titanic dataset.
class TitanicNN(nn.Module):
    def __init__(self):
        super(TitanicNN, self).__init__()
        self.layer1 = nn.Linear(7, 128)
        self.batchnorm1 = nn.BatchNorm1d(128)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.layer2 = nn.Linear(128, 64)
        self.batchnorm2 = nn.BatchNorm1d(64)
        self.layer3 = nn.Linear(64, 2)

    def forward(self, x):
        x = self.relu(self.batchnorm1(self.layer1(x)))
        x = self.dropout(x)
        x = self.relu(self.batchnorm2(self.layer2(x)))
        x = self.layer3(x)
        return x

In [51]:
# Instantiating the model
model = TitanicNN().to(device)

In [52]:
# Defining the loss function, optimizer and the learning rate scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True)

In [53]:
# Training the model
epochs = 10
for epoch in range(epochs):
    model.train()
    for data, target in train_loader:
        # Move the data and target to device
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
    
    scheduler.step()
    model.eval()
    with torch.no_grad():
        outputs = model(X_val)
        val_loss = criterion(outputs, y_val)
        _, predicted = torch.max(outputs, 1)
        correct = (predicted == y_val).sum().item()
        accuracy = correct / len(y_val)

    print(f'Epoch {epoch+1}, Loss: {loss.item()}, Val Loss: {val_loss.item()}, Accuracy: {accuracy}')

print("Training complete.")

Epoch 1, Loss: 0.39830541610717773, Val Loss: 0.5301528573036194, Accuracy: 0.7988826815642458
Epoch 2, Loss: 0.18742170929908752, Val Loss: 0.4458670914173126, Accuracy: 0.8044692737430168
Epoch 3, Loss: 0.4174325466156006, Val Loss: 0.4316645860671997, Accuracy: 0.8100558659217877
Epoch 4, Loss: 0.5852821469306946, Val Loss: 0.43834570050239563, Accuracy: 0.8100558659217877
Epoch 5, Loss: 0.34291788935661316, Val Loss: 0.44592708349227905, Accuracy: 0.8212290502793296
Epoch 6, Loss: 0.8217055201530457, Val Loss: 0.4389728605747223, Accuracy: 0.8156424581005587
Epoch 7, Loss: 0.5643500089645386, Val Loss: 0.4407312273979187, Accuracy: 0.8100558659217877
Epoch 8, Loss: 0.24520249664783478, Val Loss: 0.44080424308776855, Accuracy: 0.8100558659217877
Epoch 9, Loss: 0.545654833316803, Val Loss: 0.43657800555229187, Accuracy: 0.8156424581005587
Epoch 10, Loss: 0.593424916267395, Val Loss: 0.4440881609916687, Accuracy: 0.8156424581005587
Training complete.


In [54]:
model.eval()
with torch.no_grad():
    outputs = model(X_test)
    _, predictions = torch.max(outputs, 1)

In [55]:
test_data = pd.read_csv('./test.csv')
submission = pd.DataFrame({
    "PassengerId": test_data['PassengerId'],
    "Survived": predictions.cpu().numpy()
})
submission.to_csv('./submission.csv', index=False)

In [56]:
print(f'Validation Accuracy: {accuracy:.2f}%')

Validation Accuracy: 0.82%
